## 데이터 분포 체크

In [45]:
import json
from collections import defaultdict

# COCO 어노테이션 파일 경로
instances_file = '/Users/psjj/Downloads/coco2017/annotations/instances_train2017.json'
captions_file = '/Users/psjj/Downloads/coco2017/annotations/captions_train2017.json'

# JSON 파일 열기
with open(instances_file, 'r') as f:
    instances_data = json.load(f)

with open(captions_file, 'r') as f:
    captions_data = json.load(f)

# 카테고리 ID와 이름 매핑
category_mapping = {category['id']: category['name'] for category in instances_data['categories']}

# 클래스별 캡션 데이터 분포를 저장할 딕셔너리
class_caption_count = defaultdict(int)

# 이미지 ID와 해당 클래스 ID 매핑
image_class_mapping = defaultdict(list)
for ann in instances_data['annotations']:
    image_class_mapping[ann['image_id']].append(ann['category_id'])

# 캡션 데이터를 순회하며 해당 이미지가 어떤 클래스를 포함하는지 확인하고 카운팅
for annotation in captions_data['annotations']:
    image_id = annotation['image_id']
    caption = annotation['caption']
    
    if image_id in image_class_mapping:
        class_ids = image_class_mapping[image_id]
        for class_id in class_ids:
            class_name = category_mapping[class_id]
            class_caption_count[class_name] += 1

sorted_class_caption_count = sorted(class_caption_count.items(), key=lambda x: x[1], reverse=True)

# 정렬된 결과 출력
for class_name, count in sorted_class_caption_count:
    print(f"Class '{class_name}': {count} captions")


Class 'person': 1312984 captions
Class 'car': 219416 captions
Class 'chair': 192563 captions
Class 'book': 123644 captions
Class 'bottle': 121816 captions
Class 'cup': 103283 captions
Class 'dining table': 78622 captions
Class 'bowl': 71829 captions
Class 'traffic light': 64474 captions
Class 'handbag': 61785 captions
Class 'umbrella': 57180 captions
Class 'bird': 54077 captions
Class 'boat': 53826 captions
Class 'truck': 49885 captions
Class 'bench': 49224 captions
Class 'sheep': 47566 captions
Class 'banana': 47333 captions
Class 'kite': 45410 captions
Class 'motorcycle': 43655 captions
Class 'backpack': 43618 captions
Class 'potted plant': 43270 captions
Class 'cow': 40775 captions
Class 'wine glass': 39579 captions
Class 'carrot': 39281 captions
Class 'knife': 38881 captions
Class 'broccoli': 36565 captions
Class 'donut': 35927 captions
Class 'bicycle': 35584 captions
Class 'skis': 33245 captions
Class 'vase': 33086 captions
Class 'horse': 32937 captions
Class 'tie': 32493 captions

In [4]:
from pycocotools.coco import COCO
import requests
from PIL import Image
import os

# COCO 데이터셋 경로 설정
BASE_PATH = '/Users/psjj/Downloads/coco2017/'
annotation_file_instances = f'{BASE_PATH}annotations/instances_train2017.json'
annotation_file_captions = f'{BASE_PATH}annotations/captions_train2017.json'


# COCO API 초기화
coco = COCO(annotation_file_instances)
coco_instances = COCO(annotation_file_instances)
coco_captions = COCO(annotation_file_captions)



loading annotations into memory...
Done (t=7.13s)
creating index...
index created!
loading annotations into memory...
Done (t=8.70s)
creating index...
index created!
loading annotations into memory...
Done (t=0.56s)
creating index...
index created!


## stop sign으로 필터링 8673개

## image, json 파일 저장

In [21]:
from shutil import copyfile
import json

image_dir = f'{BASE_PATH}train2017/'
save_dir = f'{BASE_PATH}filtered_images/'
save_annotation_file = f'{BASE_PATH}filtered_captions.json'

# 필요한 디렉토리 생성
os.makedirs(save_dir, exist_ok=True)

processed_files = set(os.listdir(save_dir))  # 이미 복사된 파일 목록

# 기존에 저장된 캡션 데이터를 불러오기 (있으면)
if os.path.exists(save_annotation_file):
    with open(save_annotation_file, 'r') as f:
        filtered_annotations = json.load(f)
else:
    filtered_annotations = []
    
# desired_classes = [
#     'car', 
#     'stop sign', 
#     'traffic light', 
#     'bus', 
#     'truck', 
#     'bicycle', 
#     'bench', 
#     'fire hydrant', 
#     'motorcycle'
# ]

desired_classes = ['stop sign']
catIds = coco_instances.getCatIds(catNms=desired_classes)
imgIds = coco_instances.getImgIds(catIds=catIds)

# 이미지 및 캡션 필터링 및 저장
for img_id in imgIds:
    img_info = coco_instances.loadImgs(img_id)[0]
    img_file = os.path.join(image_dir, img_info['file_name'])
    
    # 이미 처리된 파일이면 스킵
    if img_info['file_name'] in processed_files:
        continue

    # 이미지를 새로운 폴더로 복사
    copyfile(img_file, os.path.join(save_dir, img_info['file_name']))

    # 해당 이미지의 캡션 정보 가져오기
    ann_ids = coco_captions.getAnnIds(imgIds=img_id)
    anns = coco_captions.loadAnns(ann_ids)
    
    # 캡션 데이터 저장
    for ann in anns:
        filtered_annotations.append(ann)

# 필터링된 캡션 데이터를 새로운 JSON 파일로 저장
with open(save_annotation_file, 'w') as f:
    json.dump(filtered_annotations, f)

print(f"Filtered {len(filtered_annotations)} captions and saved {len(imgIds)} images.")


Filtered 8673 captions and saved 1734 images.


In [6]:
from shutil import copyfile
import json
import os

BASE_PATH = '/Users/psjj/Downloads/coco2017/'
image_dir = f'{BASE_PATH}train2017/'
save_dir = f'{BASE_PATH}val_images/'
save_annotation_file = f'{BASE_PATH}val_captions.json'

# 필요한 디렉토리 생성
os.makedirs(save_dir, exist_ok=True)

processed_files = set(os.listdir(save_dir))  # 이미 복사된 파일 목록

# 기존에 저장된 캡션 데이터를 불러오기 (있으면)
if os.path.exists(save_annotation_file):
    with open(save_annotation_file, 'r') as f:
        filtered_annotations = json.load(f)
else:
    filtered_annotations = []

# desired_classes = [
#     'car', 
#     'stop sign', 
#     'traffic light', 
#     'bus', 
#     'truck', 
#     'bicycle', 
#     'bench', 
#     'fire hydrant', 
#     'motorcycle'
# ]

desired_classes = ['stop sign']
catIds = coco_instances.getCatIds(catNms=desired_classes)
imgIds = coco_instances.getImgIds(catIds=catIds)

# 제한된 수의 이미지 처리 (100장)
processed_count = 0
max_images = 100  # 원하는 이미지 수

# 이미지 및 캡션 필터링 및 저장
for img_id in imgIds:
    if processed_count >= max_images:  # 100장 처리 후 종료
        break

    img_info = coco_instances.loadImgs(img_id)[0]
    img_file = os.path.join(image_dir, img_info['file_name'])
    
    # 이미 처리된 파일이면 스킵
    if img_info['file_name'] in processed_files:
        continue

    # 이미지를 새로운 폴더로 복사
    copyfile(img_file, os.path.join(save_dir, img_info['file_name']))

    # 해당 이미지의 캡션 정보 가져오기
    ann_ids = coco_captions.getAnnIds(imgIds=img_id)
    anns = coco_captions.loadAnns(ann_ids)
    
    # 캡션 데이터 저장
    for ann in anns:
        filtered_annotations.append(ann)

    # 이미지 처리 카운트 증가
    processed_count += 1

# 필터링된 캡션 데이터를 새로운 JSON 파일로 저장
with open(save_annotation_file, 'w') as f:
    json.dump(filtered_annotations, f)

print(f"Filtered {len(filtered_annotations)} captions and saved {processed_count} images.")


Filtered 50 captions and saved 10 images.
